# Initialization

In [1]:
# add custom functions to path
import sys
sys.path.append("../src")

%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pickle
import json
import pandas as pd
from grabbit import Layout
from mne import read_epochs, grand_average, write_evokeds, read_evokeds
from mne import pick_types, combine_evoked, set_log_level, grand_average
from mne.time_frequency import tfr_morlet, read_tfrs, write_tfrs
from mne.viz import plot_compare_evokeds
from utils import CH_NAMES, select_subjects, drop_bad_trials
from eeg_sensor_analysis import add_events, baseline_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, fixed

sns.set(style='whitegrid', font_scale=2)

set_log_level('critical')

# load subject names to process
layout = Layout('../data', '../data/grabbit_config.json')
subjects = select_subjects(layout, 'eeg', exclude='eeg')

# load eeg configurations
with open('../config.json') as f:    
    config = json.load(f)
    
frequencies = np.arange(1, 40) 
n_cycles = frequencies

# load behavior
behavior = pd.read_csv('../data/derivatives/behavior/group_data.tsv', 
                       na_values='n/a', sep='\t') 
behavior = behavior[behavior.modality == 'eeg']

pipeline_root = '../data/derivatives/eeg_sensor'

# Make Directory Structure

In [5]:
# Make directory structure
if not os.path.exists(pipeline_root):
    os.makedirs(pipeline_root)
for subject in subjects + ['group']:
    if not os.path.exists('%s/%s' % (pipeline_root, subject)):
        os.makedirs('%s/%s' % (pipeline_root, subject))
    if not os.path.exists('%s/%s/evoked' % (pipeline_root, subject)):
        os.makedirs('%s/%s/evoked' % (pipeline_root, subject))
    if not os.path.exists('%s/%s/tfr' % (pipeline_root, subject)):
        os.makedirs('%s/%s/tfr' % (pipeline_root, subject))

# ERPs

## Make Evoked Data

In [37]:
for epo_type, epo_times in zip(config['epoch_types'], 
                               config['epoch_times']):
    print(epo_type)
    
    for event_type in config['event_types']:
        print(event_type)
        group_info = {}
        mne_evokeds = {}
        group_info['evokeds'] = {}
        group_info['naves'] = {}
        group_info['ses'] = {}
        for event in config[event_type]:
            mne_evokeds[event] = []
            group_info['evokeds'][event] = []
            group_info['ses'][event] = []
            group_info['naves'][event] = []
        group_info['bads'] = []
    
        for subject in subjects:
            print(subject)

            # load subject behavior and epochs
            sub_behavior = behavior[behavior.participant_id == subject]
            epo_file = layout.get(subject=subject, 
                                  derivative='eeg_preprocessing', 
                                  extensions='%s_cleaned-epo.fif' % epo_type)[0]
            epochs = read_epochs(epo_file.filename, verbose=False)

            # remove filter buffer 
            epochs.crop(epo_times[0], epo_times[1])
            # interpolate the bad channels
            group_info['bads'].append(epochs.info['bads'])
            epochs.interpolate_bads(reset_bads=True)
            # drop bad trials from epochs and behavior
            sub_behavior, epochs = drop_bad_trials(subject, sub_behavior, 
                                                   epochs, layout, epo_type)

            # add event labels
            epochs = add_events(epochs, sub_behavior, event_type,
                                config[event_type])

            # compute averages and standard errors 
            evos = [epochs[event].average() for event in config[event_type]]
            evos_se = [epochs[event].standard_error() 
                       for event in config[event_type]]
            
            # write subject evoked objects to file
            write_evokeds('%s/%s/%s_%s_%s-ave.fif' % (pipeline_root,
                                                      subject, subject,
                                                      event_type, epo_type), 
                          evos)
            # accumulate evoked objects for group average topomap
            for i, event in enumerate(config[event_type]):
                mne_evokeds[event].append(evos[i])
            
            # extract info for group erp object
            for i, event in enumerate(config[event_type]):
                group_info['evokeds'][event].append(evos[i].data)
                group_info['ses'][event].append(evos_se[i].data)
                group_info['naves'][event].append(evos[i].nave)
            
        # save out group erp information
        for event in config[event_type]:
            group_info['evokeds'][event] = np.array(group_info['evokeds'][event])
            group_info['ses'][event] = np.array(group_info['ses'][event])
            group_info['naves'][event] = np.array(group_info['naves'][event])
            
        group_info['chs'] = evos[0].ch_names
        group_info['times'] = evos[0].times
        f = '%s/group/group_%s_%s_erp_info.pickle' % (pipeline_root, 
                                                   event_type, epo_type)
        with open(f, 'w') as fid:
            pickle.dump(group_info, fid, protocol=pickle.HIGHEST_PROTOCOL)
            

        # calculate group average evoked object and save 
        evos = [combine_evoked(mne_evokeds[event], weights='nave')
                for event in config[event_type]]
        write_evokeds('%s/group/group_%s_%s-ave.fif' % (pipeline_root, 
                                                        event_type,
                                                        epo_type), evos)


print('Done!')

stimulus
condition
sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc014
sub-hc015
sub-hc016
sub-hc017


KeyboardInterrupt: 

## Visualize Evoked Responses 

### Plot Topomaps 

In [40]:
def plot_topomap(subject, epo_type, time, col_limit):
    plt.close('all')
    f = '%s/%s/%s_condition_%s-ave.fif' % (pipeline_root, subject, subject,
                                           epo_type)
    evokeds = read_evokeds(f)
    
    f, axs = plt.subplots(1, 4, figsize=(24, 6)) 
    
    for i, evo in enumerate(evokeds):
        evo.plot_topomap(times=time, axes=axs[i], colorbar=True, 
                         show=False, vmin=-col_limit, vmax=col_limit)
        axs[i].set_xlabel(config['condition'][i])
        
    diff = combine_evoked(evokeds, weights=[1, -1])
    diff.plot_topomap(times=time, axes=axs[2], colorbar=True, 
                      show=False, vmin=-col_limit, vmax=col_limit, show_names=True)
    axs[2].set_xlabel('i-c')
    
    plt.show();

interact(plot_topomap, subject=['group'] + subjects, 
         epo_type=['response', 'stimulus'],
         time=np.arange(-1, 1.75, .01), col_limit=np.arange(.5, 5, .5),
         diff_col_limit=np.arange(.5, 5, .5));


interactive(children=(Dropdown(description=u'subject', options=('group', 'sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016'), value='group'), Dropdown(description=u'epo_type', options=('response', 'stimulus'), value='response'), Dropdown(description=u'time', options=(-1.0, -0.98999999999999999, -0.97999999999999998, -0.96999999999999997, -0.95999999999999996, -0.94999999999999996, -0.93999999999999995, -0.92999999999999994, -0.91999999999999993, -0.90999999999999992, -0.89999999999999991, -0.8899999999999999, -0.87999999999999989, -0.86999999999999988, -0.85999999999999988, -0.84999999999999987, -0.83999999999999986, -0.82999999999999985, -0.81999999999999984, -0.80999999999999983, -0.79999999999999982, -0.78999999999999981, -0.7799999999999998, -0.7699999999999998, -0.75999999999999979, -0.74999999999999978, -0.73999999999999977, -0.72999999999999976, -0.71999999999999975, -0.70999999999999974, -0.69999999999999973, -0.68999999999999972, -0.67999999999999972, -0.66999999999999971, -0.6599999999999997, -0.64999999999999969, -0.63999999999999968, -0.62999999999999967, -0.61999999999999966, -0.60999999999999965, -0.59999999999999964, -0.58999999999999964, -0.57999999999999963, -0.56999999999999962, -0.55999999999999961, -0.5499999999999996, -0.53999999999999959, -0.52999999999999958, -0.51999999999999957, -0.50999999999999956, -0.49999999999999956, -0.48999999999999955, -0.47999999999999954, -0.46999999999999953, -0.45999999999999952, -0.44999999999999951, -0.4399999999999995, -0.42999999999999949, -0.41999999999999948, -0.40999999999999948, -0.39999999999999947, -0.38999999999999946, -0.37999999999999945, -0.36999999999999944, -0.35999999999999943, -0.34999999999999942, -0.33999999999999941, -0.3299999999999994, -0.3199999999999994, -0.30999999999999939, -0.29999999999999938, -0.28999999999999937, -0.27999999999999936, -0.26999999999999935, -0.25999999999999934, -0.24999999999999933, -0.23999999999999932, -0.22999999999999932, -0.21999999999999931, -0.2099999999999993, -0.19999999999999929, -0.18999999999999928, -0.17999999999999927, -0.16999999999999926, -0.15999999999999925, -0.14999999999999925, -0.13999999999999924, -0.12999999999999923, -0.11999999999999922, -0.10999999999999921, -0.099999999999999201, -0.089999999999999192, -0.079999999999999183, -0.069999999999999174, -0.059999999999999165, -0.049999999999999156, -0.039999999999999147, -0.029999999999999138, -0.01999999999999913, -0.0099999999999991207, 8.8817841970012523e-16, 0.010000000000000897, 0.020000000000000906, 0.030000000000000915, 0.040000000000000924, 0.050000000000000933, 0.060000000000000941, 0.07000000000000095, 0.080000000000000959, 0.090000000000000968, 0.10000000000000098, 0.11000000000000099, 0.12000000000000099, 0.130000000000001, 0.14000000000000101, 0.15000000000000102, 0.16000000000000103, 0.17000000000000104, 0.18000000000000105, 0.19000000000000106, 0.20000000000000107, 0.21000000000000107, 0.22000000000000108, 0.23000000000000109, 0.2400000000000011, 0.25000000000000111, 0.26000000000000112, 0.27000000000000113, 0.28000000000000114, 0.29000000000000115, 0.30000000000000115, 0.31000000000000116, 0.32000000000000117, 0.33000000000000118, 0.34000000000000119, 0.3500000000000012, 0.36000000000000121, 0.37000000000000122, 0.38000000000000123, 0.39000000000000123, 0.40000000000000124, 0.41000000000000125, 0.42000000000000126, 

### Plot ERP Waveforms

In [3]:
def plot_group_erps(weight, ch, behavior):
    plt.close('all')
    exclusions = ['fast_rt', 'no_response', 'error', 'post_error']
    behavior = behavior.loc[np.where(np.sum(behavior[exclusions], 
                                            axis=1) == 0)[0], :]
    fig, axs = plt.subplots(1, 2, figsize=(24, 6))
    
    for i, epo_type in enumerate(config['epoch_types']):
        ax = axs[i]
        
        f = '%s/group/group_condition_%s_erp_info.pickle' % (pipeline_root, 
                                                             epo_type)
        with open(f, 'r') as fid:
            info = pickle.load(fid)
            
        ch_ix = list(info['chs']).index(ch)
        times = info['times']
        data = info['evokeds']
        naves = info['naves']
        std_err = info['ses']
        
        for j, event in enumerate(config['condition']):
            cond_data = data[event]
            cond_data = cond_data[:, ch_ix, :] * 1e6
            
            if weight == 'equal':
                weights = np.ones(cond_data.shape[0])
            elif weight == 'nave':
                weights = naves[event]
            elif weight == 'std_err':
                weights = std_err[event]
                weights = weights[:, ch_ix, :] * 1e6
                
            cond_ave = np.average(cond_data, axis=0, weights=weights)
            variance = np.average((cond_data - cond_ave) ** 2, axis=0, 
                                   weights=weights)
            std = np.sqrt((variance * cond_data.shape[0]) / (cond_data.shape[0] - 1))
            std_error = std / np.sqrt(cond_data.shape[0]) 
            
            ax.plot(times, cond_ave, color=config['colors'][j])
            ax.fill_between(times, cond_ave - std_error, cond_ave + std_error,
                            alpha=0.5, color=config['colors'][j])
            
        for j, event in enumerate(config['condition']):
            if epo_type == 'stimulus':
                bottom=ax.get_ylim()[0]
                rt = behavior[behavior.trial_type == event].response_time
                ax.hist(rt, color=config['colors'][j], alpha=0.2, normed=True, bottom=bottom)
                
            
        if epo_type == 'stimulus':
            ax.set_xticks(np.arange(-.5, 1.8, .25))
            ax.set_xlim((-.5, 1.75))
        else:
            ax.set_xticks(np.arange(-1, 1.1, .25))
        
        ax.set_title('%s-locked' % epo_type)
        ax.axvline(0, color='k')
        ax.axhline(0, color='k')
        ax.legend(config['condition'], loc='best')
    
    plt.suptitle(ch)
    sns.despine()
    plt.show();

interact(plot_group_erps, weight=['equal', 'nave', 'std_err'], ch=CH_NAMES, behavior=fixed(behavior));

interactive(children=(Dropdown(description=u'weight', options=('equal', 'nave', 'std_err'), value='equal'), Dropdown(description=u'ch', options=('Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT9', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'FT10', 'T9', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'T10', 'TP9', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10', 'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO7', 'PO3', 'P0z', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz'), value='Fp1'), Output()), _dom_classes=('widget-interact',))

# TFR Power

## Make TFR Power

### Compute Raw TFR Power

In [3]:
for epo_type, epo_times in zip(config['epoch_types'], 
                               config['epoch_times']):
    print(epo_type)
    
    for subject in subjects:
        print(subject)

        # load subject behavior and epochs
        sub_behavior = behavior[behavior.participant_id == subject]
        epo_file = layout.get(subject=subject, 
                              derivative='eeg_preprocessing', 
                              extensions='%s_cleaned-epo.fif' % epo_type)[0]
        epochs = read_epochs(epo_file.filename, verbose=False)

        # interpolate the bad channels
        epochs.interpolate_bads(reset_bads=True)

        # drop bad trials from epochs and behavior
        sub_behavior, epochs = drop_bad_trials(subject, sub_behavior, 
                                               epochs, layout, epo_type)

        # add event labels
        epochs = add_events(epochs, sub_behavior, 'condition',
                            config['condition'])

        tfrs = []
        for event in config['condition']:
            power = tfr_morlet(epochs[event], freqs=frequencies, 
                               n_cycles=n_cycles, decim=5, 
                               return_itc=False, average=False, n_jobs=5)
            power.crop(epo_times[0], epo_times[1])
            tfrs.append(power)

        f = '../data/derivatives/eeg_sensor/%s/tfr/%s_%s-locked_raw-tfr.h5'
        write_tfrs(f % (subject, subject, epo_type), tfrs, overwrite=True)

print('Done!')

stimulus
sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc014
sub-hc015
sub-hc016
sub-hc017
sub-hc019
sub-hc020
sub-hc021
sub-hc022
sub-hc023
sub-hc024
sub-hc025
sub-hc026
sub-hc028
sub-hc029
sub-hc030
sub-hc031
sub-hc032
sub-hc033
sub-hc034
sub-hc035
sub-hc036
sub-hc037
sub-hc042
sub-hc044
sub-hc045
sub-pp001
sub-pp002
sub-pp003
sub-pp004
sub-pp005
sub-pp006
sub-pp007
sub-pp008
sub-pp009
sub-pp010
sub-pp011
sub-pp012
sub-pp013
sub-pp014
sub-pp015
sub-pp016
response
sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc014
sub-hc015
sub-hc016
sub-hc017
sub-hc019
sub-hc020
sub-hc021
sub-hc022
sub-hc023
sub-hc024
sub-hc025
sub-hc026
sub-hc028
sub-hc029
sub-hc030
sub-hc031
sub-hc032
sub-hc033
sub-hc034
sub-hc035
sub-hc036
sub-hc037
sub-hc042
sub-hc044
sub-hc045
sub-pp001
sub-pp002
sub-pp003
sub-pp004
sub-pp005
sub-pp006
sub-pp007
sub-pp008
su

### Baseline Normalize Power

In [6]:
for epo_type, epo_times in zip(config['epoch_types'], 
                               config['epoch_times']):
    print(epo_type)
    
    baselines = {}
    group = {'incongruent': [], 'congruent': []}
    for subject in subjects:
        print(subject)
        
        f = '../data/derivatives/eeg_sensor/%s/tfr/%s_%s-locked_raw-tfr.h5'
        tfrs = read_tfrs(f % (subject, subject, epo_type))
        
        if epo_type == 'stimulus':
            baselines[subject] = {'incongruent': (-.5, -.1), 'congruent': (-.5, -.1)}
        
        norm_tfrs = []
        for i, c in enumerate(config['condition']):
            
            tfr, baseline = baseline_normalize(tfrs[i], baselines[c])
            norm_tfrs.append(tfr)
            group[c].append(tfr)
            baselines[subject][c] = baseline
            
            
        f = '../data/derivatives/eeg_sensor/%s/tfr/%s_%s-locked_norm-tfr.h5'
        write_tfrs(f % (subject, subject, epo_type), norm_tfrs, 
                   overwrite=True)
    group_tfrs = [grand_average(group[c]) for c in config['condition']]
    f = '../data/derivatives/eeg_sensor/group/tfr/group_%s-locked_norm-tfr.h5'
    write_tfrs(f % (epo_type), group_tfrs, overwrite=True)
        
print('Done!')

stimulus
sub-hc001


KeyError: u'incongruent'

## Visualize TFR Power

### TFR Power Heatmaps

In [5]:
def plot_tfr_heatmap(subject, ch, lim):
    exclusions = ['fast_rt', 'no_response', 'error', 'post_error']
    sub_behavior = behavior.loc[np.where(np.sum(behavior[exclusions], 
                                            axis=1) == 0)[0], :]
    if subject != 'group':
        sub_behavior = sub_behavior.loc[sub_behavior.participant_id == subject, :]
    sns.set(style='white', font_scale=2)
    plt.close('all')

    fig, axs = plt.subplots(2, 3, figsize=(24, 16))
    
    
    for i, epo_type in enumerate(config['epoch_types']):
        
        f = '../data/derivatives/eeg_sensor/%s/tfr/%s_%s-locked_norm-tfr.h5'
        tfrs = read_tfrs(f % (subject, subject, epo_type))

        powers = []
        for j, c in enumerate(config['condition']):

            power = tfrs[j]
            power = power.pick_channels([ch])
            powers.append(power)
            times = power.times
            freqs = power.freqs

            ax = axs[i, j]
            ax.contourf(power.times, power.freqs, power.data.squeeze(), 40, 
                        cmap='jet', vmin=-lim, vmax=lim)
            
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('Frequency (Hz)')
            ax.set_title('%s %s-locked' % (c, epo_type))
            ax.axvline(0, color='k')
            for v in [4, 8, 15, 30]:
                ax.axhline(v, color='k', linestyle='--')
            
            if epo_type == 'stimulus':
                rt = sub_behavior[sub_behavior.trial_type == c].response_time
                ax.hist(rt, color=config['colors'][j],
                            normed=True, bottom=-4)
                

        ax = axs[i, 2]
        ax.contourf(power.times, power.freqs, 
                    (powers[0].data.squeeze() - powers[1].data.squeeze()), 40, 
                    cmap='jet', vmin=-lim, vmax=lim)

        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Frequency (Hz)')
        ax.set_title('i - c %s-locked' % (epo_type))
        ax.axvline(0, color='k')
        for v in [4, 8, 15, 30]:
            ax.axhline(v, color='k', linestyle='--')
        for j, c in enumerate(config['condition']):
            if epo_type == 'stimulus':
                rt = sub_behavior[sub_behavior.trial_type == c].response_time
                ax.hist(rt, color=config['colors'][j], alpha=0.5,
                            normed=True, bottom=-4)
        
    plt.tight_layout()
    plt.subplots_adjust(top=.92)
    plt.suptitle('%s %s TFR Heatmaps' % (subject, ch), fontsize=24)
    plt.show()

interact(plot_tfr_heatmap, subject=['group'] + subjects, ch=CH_NAMES, 
         lim=(2));

interactive(children=(Dropdown(description=u'subject', options=('group', 'sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016'), value='group'), Dropdown(description=u'ch', options=('Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT9', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'FT10', 'T9', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'T10', 'TP9', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10', 'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO7', 'PO3', 'P0z', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz'), value='Fp1'), IntSlider(value=2, description=u'lim', max=6, min=-2), Output()), _dom_classes=('widget-interact',))

### TFR Power Topomaps